# 09-1. 순차 데이터와 순환 신경망
> p.486
- 나중에 서비스화 하기에는 조금 거리가 있는 내용이긴함.


# 09-2. 순환 신경망으로 IMDB 리뷰 분류하기
> p. 500
- 텐서플로를 사용해 순환 신경망을 만들어 영화 리뷰 데이터셋에 적용해서 리뷰를 긍정과 부정으로 분류하기
  > https://developer.imdb.com/documentation/api-documentation/calling-the-api/
  - 영화 리뷰 데이터 api: 이 연습에서는 총 50,000개로 이루어진 긍정/ 부정 (각 25,000개) 으로 분류된 리뷰 데이터를 사용함.

  > `순환 신경망` 분류 과정: 각 단어를 숫자로 바꾼다. 그리고 나뉜 단어(숫자)를 점수를 매겨 긍정/ 부정을 나눈다.

In [3]:
# 텐서플로우에 저장되어있는 IMDB 리뷰 데이터 불러오기

from tensorflow import keras

In [4]:
(train_input, train_target), (test_input, test_target) = keras.datasets.imdb.load_data(num_words=300)

In [5]:
train_input.shape

(25000,)

In [5]:
print(train_input.shape, test_input.shape)

(25000,) (25000,)


In [6]:
len(train_input[0])
# 리뷰를 문자가 아닌 숫자로 바라 볼 것이다.
# 이미 숫자로 치환된 값을 볼 것이다.

# 2라는 숫자는, 자주 쓰지 않은 단어로, 내가 쓰는 데이터에는 불필요하다고 2로 처리한 데이터값이다.

# 첫 번째 리뷰의 길이는 218개의 토큰으로 이루어져있다.

218

In [7]:
# 우리가 맞춰야되는 정보를 확인해보자
train_target[:10]
# 첫번째 리뷰는 긍정 (1), 두번째 리뷰는 부정 (0) 이다 => 이 데이터셋을 만든 사람이 이미 1은 긍정, 0은 부정이라고 분류해둔 상황

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [6]:
from sklearn.model_selection import train_test_split

train_input, val_input, train_target, val_target = train_test_split(train_input, train_target)

In [7]:
# 모든 리뷰데이터를 일괄적으로 같은 규격으로 맞춰줘야한다. (p.506)
# pad_sequences

from keras.preprocessing.sequence import pad_sequences
train_seq = pad_sequences(train_input, maxlen=100)

In [12]:
train_seq.shape
# 우리가 가진 데이터 중에 일부분은 테스트용으로 뺐고,
# 100단어로 규격을 맞춰줌.
# 학습을 시킬 때, 일괄적으로 똑같은 규격이 있어야 하기 때문에

(18750, 100)

In [11]:
train_seq[0]
# 자른 이후의 데이터

array([ 21, 115,   2,   4, 229, 115,   2,   2, 101, 109,  24,  60,   4,
         2,   2,  12,  82,   5,   2,  39,   2,  23,   2,   2,   2,   2,
         5,   2,   4, 192,  15,   2,   9,  38,   2,  11,   4,  22,   5,
         9,  53,   2,   2,  74,  59,  47, 126,  77,   9,   6,   2,   8,
        72,  13, 106,   2,  33, 222, 280,   6, 291,   5,  13,  43,   2,
        12, 174,  14,   2,   2,  19,  61,   2, 291, 154,   2,   2,   5,
        36,   2,  12,  17,  73,  12,  47, 142,  18,   2,  19,   6,   2,
         5,   2,  25,   2,  19,   2,  11,   4, 130], dtype=int32)

In [13]:
train_input[0][-10:]
# 자르기 전 데이터
# 뒷 부분이 같다.

[34, 4, 107, 293, 156, 4, 2, 5, 2, 2]

In [8]:
val_seq = pad_sequences(val_input, maxlen=100)

In [14]:
val_seq.shape
# 검증용 데이터까지 준비 완료

(6250, 100)

In [27]:
# 깡통 모델 만들기
# 인스턴스화
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(8, input_shape=(100, 300)))
# meaning: 100개의 토큰을 가진 데이터 하나가, 300개의 단어로 이루어진 숫자야
model.add(keras.layers.Dense(1, activation='sigmoid'))
# 0 혹은 1의 이진 분류를 위해
# Dense(1)을 부여함.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
# 위 코드를 통해 모델이 성공적으로 만들어졌다.
# 원핫인코딩 적용하기
train_oh = keras.utils.to_categorical(train_seq)

In [17]:
train_oh.shape
# 단어 하나를 표현하기 위해서 300개 짜리 칸을 만듦.

(18750, 100, 300)

In [18]:
train_oh[0][0]
 # 300개 짜리 칸에서 세번째만 1: 3을 의미

 # 숫자간의 우위가 없다.
 # 1차원이었던 숫자 데이터를 2차원으로 바꿔준 것.

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [19]:
# val데이터도 동일하게 원핫인코딩을 해줘야함.
val_oh = keras.utils.to_categorical(val_seq)

In [20]:
# 데이터 준비가 완료 되었다.

In [21]:
 # 만든 모델을 확인하자.
 model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 8)                   │           2,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,481 (9.69 KB)

 Trainable params: 2,481 (9.69 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# 만든 모델을 사용해서
# 학습을 시키자. (p.512)
# optimizer의 값을 조금 조정해서 학습을 시키겠다. (책에서 이 값을 조정했을 때, 성능이 더 좋더라라고 조언을 해주었기에)
# rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
# model.compile(
#     optimizer=rmsprop,
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# ) # 이진분류이기 때문에 loss 함수를 binary로 적었음.
# es = keras.callbacks.EarlyStopping(patience=3)
# result = model.fit(
#     train_oh,
#     train_target,
#     epochs=50,
#     validation_data=(val_oh, val_target),
#     callbacks=[es])

지금 RAM이 부족해서 위 코드를 실행 하지 못함.
>
=> 따라서 코드 결과는 책에서 확인하고 넘어가는 걸로 (p.514)

#### 단어 임베딩을 사용하기 (p. 514)
> 순환 신경망에서 텍스트를 처리할 때 즐겨 사용하는 방법
>> 각 단어를 고정된 크기의 실수 벡터로 바꾸어 줌.
: 벡터는 원-핫 인코딩된 벡터보다 훨씬 의미 있는 값으로 채워져 있기 때문에 자연어 처리에서 더 좋은 성능을 냄.

: `장점` - 입력으로 정수 데이터를 받는다. (연산이 가능하다.)

: `임베딩`은 보통 다 이미 학습 (구현) 이 되어 있다. (모델별로 수치는 다르나)

In [9]:
# 새로운 모델 만들기
model2 = keras.Sequential()
model2.add(keras.layers.Embedding(300, 16, input_length=100))
model2.add(keras.layers.SimpleRNN(8))
model2.add(keras.layers.Dense(1, activation = 'sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [41]:
print(train_seq.shape, val_seq.shape)

(18750, 100) (6250, 100)


In [14]:
model2.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
es = keras.callbacks.EarlyStopping(patience=2)
result = model2.fit(
    train_seq,
    train_target,
    epochs=100,
    validation_data=(val_seq, val_target),
    callbacks=[es]
)

Epoch 1/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.7029 - loss: 0.5861 - val_accuracy: 0.6453 - val_loss: 0.6366
Epoch 2/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.7182 - loss: 0.5738 - val_accuracy: 0.6973 - val_loss: 0.5946
Epoch 3/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7386 - loss: 0.5444 - val_accuracy: 0.6845 - val_loss: 0.5896
Epoch 4/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7354 - loss: 0.5412 - val_accuracy: 0.6984 - val_loss: 0.6155
Epoch 5/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7126 - loss: 0.5732 - val_accuracy: 0.7088 - val_loss: 0.5747
Epoch 6/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7401 - loss: 0.5355 - val_accuracy: 0.7224 - val_loss: 0.5645
Epoch 7/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.7368 - loss: 0.5374 - val_accuracy: 0.7128 - val_loss: 0.5654
Epoch 8/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7264 - loss: 0.5403 

## 09-3. LSTM과 GRU 셀
> (p.526)

> 코드 따로 치지 않고 교재 내용으로 살펴봄.

### 🔹 GRU(Gated Recurrent Unit) & LSTM(Long Short-Term Memory)

GRU와 LSTM은 **순환 신경망(RNN)의 단점**을 보완한 **개선된 RNN 모델**이다.  
특히, **장기 의존성 문제**(Long-Term Dependency)와 **기울기 소실 문제**(Vanishing Gradient)를 해결하기 위해 개발되었다.

---

### 🔹  LSTM (Long Short-Term Memory)

LSTM은 **장기 메모리(Long-Term Memory)와 단기 메모리(Short-Term Memory)**를 조절하는 **3개의 게이트(Gate)**를 가지고 있다.

#### 🔸 LSTM 구조
- **입력 게이트 (Input Gate)**: 새로운 정보를 얼마나 업데이트할지 결정  
- **망각 게이트 (Forget Gate)**: 기존 정보를 얼마나 버릴지 결정  
- **출력 게이트 (Output Gate)**: 최종적으로 어떤 정보를 출력할지 결정  

#### 📌 LSTM의 특징  
- ✅ **장기 의존성 문제 해결** → 중요한 정보를 오랫동안 유지 가능  
- ✅ **복잡한 패턴 학습 가능**  
- ❌ **연산량이 많고 속도가 느림**  

---

### 🔹  GRU (Gated Recurrent Unit)

GRU는 **LSTM보다 간단한 구조**로, 연산량을 줄이면서도 **장기 의존성 문제**를 해결한 모델이다.

#### 🔸 GRU 구조
- **업데이트 게이트 (Update Gate)**: 새로운 정보를 업데이트할지 결정  
- **리셋 게이트 (Reset Gate)**: 과거 정보를 얼마나 유지할지 결정  

#### 📌 GRU의 특징  
- ✅ **LSTM보다 연산량이 적어 속도가 빠름**  
- ✅ **메모리 사용량이 적음**  
- ❌ **LSTM보다 세밀한 제어가 어려울 수 있음**  

---

#### 🔹  LSTM vs GRU 비교

| 모델 | 특징 | 장점 | 단점 |
|------|------|------|------|
| **LSTM** | 3개의 게이트 (입력, 망각, 출력) 사용 | 장기 의존성 문제 해결, 복잡한 패턴 학습 가능 | 연산량 많고 속도 느림 |
| **GRU** | 2개의 게이트 (업데이트, 리셋) 사용 | 연산량 적고 학습 속도 빠름, 메모리 절약 | 세밀한 조절이 어려울 수 있음 |

---

#### 🔹  LSTM과 GRU 선택 기준

- **빠른 학습 & 적은 메모리 사용** → **GRU**  
- **정확도 우선 & 장기 기억 필요** → **LSTM**  
